# electricity_market_env

> Fill in a module description here

In [ ]:
#| default_exp electricity_market_env

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import gymnasium as gym
import numpy as np

In [ ]:
class ElectricityMarketEnv(gym.Env):
    def __init__(self, env_config):
        # Environment Configuration
        self._battery_capacity = env_config['battery_capacity']
        self._config = env_config

        self.action_space = gym.spaces.Box(low=-self._battery_capacity, high=self._battery_capacity, shape=(1,), dtype=np.float64)
        self.observation_space = gym.spaces.Box(low=np.array([0, 0, 0]),
                                                high=np.array([self._battery_capacity, np.inf, np.inf]),
                                                shape=(3,), dtype=np.float64)


        # State of Environment
        self._current_state_of_charge = env_config['init_state_of_charge']
        self._timestep = 0

    def _is_action_valid(self, action) -> bool:
        target_state_of_charge = self._current_state_of_charge + action
        return target_state_of_charge > self._battery_capacity or target_state_of_charge < 0

    def step(self, action):
        if not self._is_action_valid(action):
            raise ValueError('Invalid action')

        reward = self._calculate_reward(action)

        self._current_state_of_charge += action
        self._timestep += 1

        done = False
        observation = self._get_obs()

        return observation, reward, done, {}

    def _calculate_reward(self, action) -> float:
        return  (action - self._demand_of_electricity()) * self._price()

    def _demand_of_electricity(self) -> float:
        noise_std = 5
        demand = 120 * np.exp(-((self._timestep - 0.4) ** 2) / (2 * (0.05 ** 2))) + 100 * np.exp(-((self._timestep - 0.7) ** 2) / (2 * (0.1 ** 2)))
        noise = np.random.normal(0, noise_std)
        return demand + noise


    def _price(self) -> float:
        noise_std = 5
        f1 = 2
        f2 = 3
        price = 100 * np.sin(2 * np.pi * f1 * self._timestep) + 50 * np.sin(2 * np.pi * f2 * self._timestep)
        noise = np.random.normal(0, noise_std)
        return price + noise

    def reset(self, seed=None, options=None):
        # We need the following line to seed self.np_random
        super().reset(seed=seed)

        self._current_state_of_charge = self._config['init_state_of_charge']
        self._timestep = 0

        observation = self._get_obs()
        return observation

    def _get_obs(self):
        return np.array([self._current_state_of_charge, self._demand_of_electricity(), self._price()])



In [ ]:
#| hide
env = ElectricityMarketEnv({
    "battery_capacity": 100,
    "init_state_of_charge": 100,
    "init_current_demand_of_electricity": 0,
    "init_current_price": 10
})

env.reset()

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()